<a href="https://colab.research.google.com/github/Pallavilandge04/prompt-engineering-class/blob/main/projectfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain chromadb sentence-transformers langchain-community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.0 MB/s eta 0:00:0

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import os

# Read the text file
with open('/content/healthcare_dataset2.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Split the text into paragraphs
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)
paragraphs = text_splitter.split_text(text)

# Initialize the embedding model
embeddings = HuggingFaceEmbeddings()

# Create and persist the vector store
persist_directory = 'db'
vectorstore = Chroma.from_texts(texts=paragraphs, embedding=embeddings, persist_directory=persist_directory)
vectorstore.persist()

print(f"Created and persisted vector store with {len(paragraphs)} paragraphs.")


<ipython-input-17-88979afa9b2e>:20: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


Created and persisted vector store with 14 paragraphs.


In [ ]:
pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
pwd

'/content'

In [18]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Set your Groq API key
os.environ["GROQ_API_KEY"] = "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"


# Initialize the Groq language model
llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0)

# Create a custom prompt template
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Create the RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# Function to answer questions
def answer_question(question):
    result = qa({"query": question})
    answer = result['result']
    sources = [doc.page_content for doc in result['source_documents']]
    return answer, sources

# Example usage

question = "tell me more about condition angina?"
answer, sources = answer_question(question)
print(f"Question: {question}")
print(f"Answer: {answer}\n")



Question: tell me more about condition angina?
Answer: I'm sorry for any confusion, but the provided clinical notes do not include any information about a patient with angina. Angina is a term used to describe chest pain or discomfort caused by an inadequate supply of oxygen-rich blood to the heart muscle. It's often a symptom of coronary artery disease. If you have more specific notes or context related to angina, I would be happy to help you understand it better.

